In [0]:
import bs4 as bs
import datetime as dt
import os
from pandas_datareader import data as pdr
import pickle
import requests
import fix_yahoo_finance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

yf.pdr_override

<function fix_yahoo_finance.pdr_override>

In [0]:
def save_nifty_50_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/NIFTY_50')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text
        tickers.append(ticker)
        
    with open("nifty50tickers_org.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers
    

save_nifty_50_tickers()

['ADANIPORTS.NS',
 'ASIANPAINT.NS',
 'AXISBANK.NS',
 'BAJAJ-AUTO.NS',
 'BAJFINANCE.NS',
 'BAJAJFINSV.NS',
 'BHARTIARTL.NS',
 'INFRATEL.NS',
 'BPCL.NS',
 'BRITANNIA.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'DRREDDY.NS',
 'EICHERMOT.NS',
 'GAIL.NS',
 'GRASIM.NS',
 'HCLTECH.NS',
 'HDFC.NS',
 'HDFCBANK.NS',
 'HEROMOTOCO.NS',
 'HINDALCO.NS',
 'HINDUNILVR.NS',
 'ICICIBANK.NS',
 'INDUSINDBK.NS',
 'INFY.NS',
 'IOC.NS',
 'ITC.NS',
 'JSWSTEEL.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'M&M.NS',
 'MARUTI.NS',
 'NESTLEIND.NS',
 'NTPC.NS',
 'ONGC.NS',
 'POWERGRID.NS',
 'RELIANCE.NS',
 'SHREECEM.NS',
 'SBIN.NS',
 'SUNPHARMA.NS',
 'TCS.NS',
 'TATAMOTORS.NS',
 'TATASTEEL.NS',
 'TECHM.NS',
 'TITAN.NS',
 'ULTRACEMCO.NS',
 'UPL.NS',
 'VEDL.NS',
 'WIPRO.NS',
 'ZEEL.NS']

In [0]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("nifty50tickers_org.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('nifty_dfs_test1'):
        os.makedirs('nifty_dfs_test1')

    start = dt.datetime(2010, 1, 1)
    end = dt.datetime.now()
    for ticker in tickers:
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('nifty_dfs_test1/{}.csv'.format(ticker)):
            df = pdr.get_data_yahoo(ticker, start, end)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            #df = df.drop("Symbol", axis=1)
            df.to_csv('nifty_dfs_test1/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))


get_data_from_yahoo()

In [0]:
def compile_data():
    with open("nifty50tickers_org.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df_org = pd.DataFrame()

    for count, ticker in enumerate(tickers):
        df = pd.read_csv('nifty_dfs_test1/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close': ticker}, inplace=True)
        df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)

        if main_df_org.empty:
            main_df_org = df
        else:
            main_df_org = main_df_org.join(df, how='outer')

        if count % 10 == 0:
            print(count)
    print(main_df_org.head())
    main_df_org.to_csv('sp500_joined_closes_org.csv')


compile_data()

0
10
20
30
40
            ADANIPORTS.NS  ASIANPAINT.NS  ...    WIPRO.NS     ZEEL.NS
Date                                      ...                        
2010-01-04     102.461502     112.306633  ...  107.914261  106.504539
2010-01-05     105.994957     112.661400  ...  109.617630  107.664505
2010-01-06     111.249168     112.322319  ...  107.401352  110.047379
2010-01-07     109.933304     111.176407  ...  105.495438  110.111267
2010-01-08     109.804497     113.012978  ...  103.908943  108.466278

[5 rows x 50 columns]


In [0]:
from collections import Counter
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500_joined_closes_org.csv', index_col=0)
    tickers = df.columns.values.tolist()
    df.fillna(0, inplace=True)

    for i in range(1,hm_days+1):
        df['{}_{}d'.format(ticker,i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]

    df.fillna(0, inplace=True)
    return tickers, df
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.02
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0


def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)

    df['{}_target'.format(ticker)] = list(map( buy_sell_hold,
                                               df['{}_1d'.format(ticker)],
                                               df['{}_2d'.format(ticker)],
                                               df['{}_3d'.format(ticker)],
                                               df['{}_4d'.format(ticker)],
                                               df['{}_5d'.format(ticker)],
                                               df['{}_6d'.format(ticker)],
                                               df['{}_7d'.format(ticker)]))

    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:', Counter(str_vals))

    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)

    df_vals = df[[ticker for ticker in tickers]].pct_change()
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)

    X = df_vals.values
    y = df['{}_target'.format(ticker)].values
    return X, y, df

In [0]:
from statistics import mean
import pickle
import pandas as pd
import numpy as np
from sklearn import svm,neighbors
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier

def do_ml(ticker):
    X, y, df = extract_featuresets(ticker)

    #X_train, X_test, y_train, y_test = cross_val_score.train_test_split(X, y, test_size=0.25)
    X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25)

    clf = VotingClassifier([('lsvc', svm.LinearSVC()),
                            ('knn', neighbors.KNeighborsClassifier()),
                            ('rfor', RandomForestClassifier())])

    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print('accuracy:', confidence)
    predictions = clf.predict(X_test)
    print('predicted class counts:', Counter(predictions))
    print()
    print()
    return confidence



# examples of running:
#do_ml('CIPLA.NS')
#do_ml('AAPL')
#do_ml('ABT')
with open("nifty50tickers_org.pickle","rb") as f:
    tickers = pickle.load(f)

accuracies = []
for count,ticker in enumerate(tickers):

    if count%10==0:
        print(count)

    accuracy = do_ml(ticker)
    accuracies.append(accuracy)
    print("{} accuracy: {}. Average accuracy:{}".format(ticker,accuracy,mean(accuracies)))

0
Data spread: Counter({'1': 1242, '-1': 1158, '0': 138})
accuracy: 0.462992125984252
predicted class counts: Counter({1: 398, -1: 237})


ADANIPORTS.NS accuracy: 0.462992125984252. Average accuracy:0.462992125984252
Data spread: Counter({'1': 1191, '-1': 905, '0': 442})
accuracy: 0.45354330708661417
predicted class counts: Counter({1: 536, -1: 98, 0: 1})


ASIANPAINT.NS accuracy: 0.45354330708661417. Average accuracy:0.4582677165354331
Data spread: Counter({'1': 1227, '-1': 1100, '0': 211})
accuracy: 0.47716535433070867
predicted class counts: Counter({1: 453, -1: 182})


AXISBANK.NS accuracy: 0.47716535433070867. Average accuracy:0.4645669291338583
Data spread: Counter({'0': 961, '1': 835, '-1': 742})
accuracy: 0.4628751974723539
predicted class counts: Counter({-1: 273, 0: 266, 1: 94})


BAJAJ-AUTO.NS accuracy: 0.4628751974723539. Average accuracy:0.4641439962184822
Data spread: Counter({'1': 1381, '-1': 941, '0': 216})
accuracy: 0.49921259842519683
predicted class counts: Counter({

# New Section